In [17]:
import torch
import torch.nn as nn
from torch.utils.data import ConcatDataset, DataLoader
from model import FaceDetector
from yoloy_dataset import YoloDataset
from loss import Yolo_Loss
import os

In [18]:
model = FaceDetector()

/home/kuba/.virenv/base/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kuba/.virenv/base/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
PATH = '/home/kuba/Documents/data/raw/single-face-tensors/train'
all_train_datasets = []

for file in sorted(os.listdir(PATH)):
    try:
        dataset = YoloDataset((PATH+"/"+file))
        all_train_datasets.append(dataset)
    except Exception as e:
        print(f"Error loading {file}: {str(e)}")
        continue

combined = ConcatDataset(all_train_datasets)

In [20]:
optimizer = torch.optim.Adam(model.regression_head.parameters(), lr=0.001)
criterion = Yolo_Loss()

/home/kuba/.virenv/base/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [21]:
train_loader = DataLoader(combined, batch_size=32)

In [22]:
len(combined)

17373

In [23]:
X, y = next(iter(train_loader))

In [24]:
X.shape

torch.Size([32, 3, 224, 224])

In [29]:
out = model(X)
loss = criterion(out, y)
optimizer.zero_grad()
loss.backward()
optimizer.step()
loss.item()

670412032.0